only one window?



In [ ]:
var importer = require('../Core');

var closeAllTabs = (keep) => {
    return client
        .getTabIds()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.length; i++) {
                if(h[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .switchTab(h[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
}

var closeAllWindows = (keep) => {
    return client.windowHandles()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                if(h.value[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .window(h.value[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .window(keep)
}

var onlyOneWindow = () => {
    return client
        .windowHandles()
        .then(r => client.window(r.value[0]))
        .newWindow('https://google.com')
        .windowHandle()
        .then(r => closeAllWindows(r.value))
        .getCurrentTabId()
        .then(r => closeAllTabs(r))
    // TODO: close all tabs
}
module.exports = onlyOneWindow;



tile selenium chrome windows?



In [ ]:
var importer = require('../Core');

const rows = 6;
const columns = 6;
var screen;

var openUrl = (url) => {
    var handles;
    return client
        .windowHandles()
        .then(h => handles = h.value)
        .execute((url, width, height) => {
            window.open(url, '', 'width=' + width + ',height=' + height);
        }, url, Math.floor(screen.width / columns), Math.floor(screen.height / rows))
        .windowHandles()
        .then(h => h.value.filter(hwnd => handles.indexOf(hwnd) === -1)[0])
        .catch(e => console.log(e))
    /*
    // TODO: use this openSPA()
    var id = Math.random().toString(36).substring(7);
    return client
        .execute((url, id, width, height) => {
            var a = document.createElement('a');
            a.setAttribute('onclick', 'window.open("' + url + '", "", "width=' + width + ',height=' + height + '"); return false;');
            a.setAttribute('id', id);
            a.href = url;
            a.style.position = 'absolute';
            a.style.zIndex = 4294967295;
            a.style.top = 0;
            a.style.left = 0;
            a.style.bottom = 0;
            a.style.right = 0;
            document.body.appendChild(a);
            setTimeout(() => a.remove(), 200);
        }, url, id, Math.floor(screen.width / columns), Math.floor(screen.height / rows))
        .click('a[id="' + id + '"]')
        */
}

var createNewWindows = (urls) => {
    var hwnd, handles;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandles()
        .then(r => handles = r.value)
        .then(() => {
            // -1 first window is already used by sliced url
            const count = urls.length - (handles.length - 1);
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    return openUrl(urls[i])
                             .window(hwnd)
                             .then(h => resolve(h))
                             .catch(e => { console.log(e); resolve() })
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .windowHandles()
        .then(r => handles = r.value)
        // position all windows after opening
        .then(() => {
            const promises = [];
            for(var i = 0; i < handles.length; i++) {
                promises.push((i => resolve => {
                    const newPosition = {
                        x: Math.floor(i % columns) * Math.floor(screen.width / columns),
                        y: Math.floor(i / columns % rows) * Math.floor(screen.height / rows)
                    };
                    console.log(newPosition);
                    return client
                        .window(handles[i])
                        .windowHandlePosition(newPosition)
                        .then(() => resolve())
                        .catch(e => {console.log(e); resolve()})
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var tileWindows = (urls) => {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandleMaximize(hwnd)
        .windowHandleSize()
        .then(r => screen = r.value)
        .windowHandlePosition({x: 0, y: 0})
        .then(() => client.windowHandleSize({
            width: Math.floor(screen.width / columns),
            height: Math.floor(screen.height / rows)
        }))
        .url(typeof urls === 'string' ? urls : urls[0])
        .then(h => typeof urls === 'string' ? [] : createNewWindows(urls.slice(1)))
        .windowHandles()
        .catch(e => console.log(e))
}
module.exports = tileWindows;



In [7]:
var importer = require('../Core');

var query = 'list of search engines';

$$.async();
importer.import('selenium cell')
    .then(runSeleniumCell => runSeleniumCell('tile chrome windows'))
    .then(tileWindows => tileWindows([
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'http://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://archive.org/search.php?query=' + query,
    ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



Initializing webdriver on localhost
{ Error: A session is either terminated or not started
    at session() - index.js:232:23
  message: 'No active session with ID 4cbe67154f5565371bc073f0331e31b2',
  type: 'RuntimeError',
  seleniumStack: 
   { status: 6,
     type: 'NoSuchDriver',
     message: 'A session is either terminated or not started',
     orgStatusMessage: 'No active session with ID 4cbe67154f5565371bc073f0331e31b2' } }
[ null, '493b6f59e96f0216f6462b294752228b' ]
[ '493b6f59e96f0216f6462b294752228b' ]
{ x: 0, y: 0 }
{ x: 1360, y: 0 }
{ x: 2720, y: 0 }
{ x: 4080, y: 0 }
{ x: 5440, y: 0 }
{ x: 6800, y: 0 }
{ x: 0, y: 1016 }
{ x: 1360, y: 1016 }
{ x: 2720, y: 1016 }


{ sessionId: '493b6f59e96f0216f6462b294752228b',
  status: 0,
  value: 
   [ 'CDwindow-1bb89641-72db-41c2-915a-e17b5a71557d',
     'CDwindow-5cec8ff2-4054-42b6-8ac9-86d89d0180a0',
     'CDwindow-a5c56c06-d15a-477f-83e1-5ca7b5d5538d',
     'CDwindow-0c93931c-f815-436b-98f3-aa2971b70d79',
     'CDwindow-472e51c7-9a4e-4bef-ad06-ebb64bb3a9d0',
     'CDwindow-e595bac8-b804-4b8d-9f41-b6aa2c9cc56f',
     'CDwindow-111fa438-f1c4-437a-9f14-40c97c380c90',
     'CDwindow-eccb38bc-3fef-4090-b69d-3db979ca029f',
     'CDwindow-1bae73c9-7880-4cfd-86dd-39a21153344b' ] }